In [2]:
import pandas as pd
import networkx as nx 
from pyvis.network import Network
import time

pd.options.display.max_columns = 50

c:\DEV\knowledge_graph_hh\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Загрузка данных


In [272]:
df_raw_raw = pd.read_csv(r"D:\hh_2006-2020\vacancies_2016.csv")
display(df_raw_raw.head(3))
display(df_raw_raw.shape)

,id,description,key_skills,schedule_id,schedule_name,accept_handicapped,accept_kids,experience_id,experience_name,specializations,contacts,billing_type_id,billing_type_name,allow_messages,premium,driver_license_types,accept_incomplete_resumes,employer_id,employer_name,employer_vacancies_url,employer_trusted,employer_alternate_url,employer_industries,response_letter_required,type_id,...,salary_gross,salary_currency,archived,name,insider_interview,area_id,area_name,area_url,created_at,published_at,address_city,address_street,address_building,address_description,address_lat,address_lng,alternate_url,apply_alternate_url,code,department_id,department_name,employment_id,employment_name,prof_classes_found,terms_found
0,16260123,<p>Современная и амбициозная российская компан...,UML\nUse Case Analysis\nнаписание технических ...,fullDay,Полный день,False,False,between1And3,От 1 года до 3 лет,"1.221 Программирование, Разработка 1 Информаци...",NaN,standard,Стандарт,True,False,NaN,False,707239.0,Embria,https://api.hh.ru/vacancies?employer_id=707239,True,https://hh.ru/employer/707239,"Интернет-компания (поисковики, платежные систе...",False,open,...,NaN,NaN,True,Системный аналитик,NaN,2,Санкт-Петербург,https://api.hh.ru/areas/2,2016-03-10T20:56:17+0300,2016-03-10T20:56:17+0300,NaN,NaN,NaN,NaN,NaN,NaN,https://hh.ru/vacancy/16260123,https://hh.ru/applicant/vacancy_response?vacan...,NaN,NaN,NaN,full,Полная занятость,analyst,техническое_образование конкурентная_зарплата ...
1,16260126,"<p><em><strong>Развивающаяся компания, успешно...",Активные продажи\nХолодные продажи\nВедение пе...,fullDay,Полный день,False,False,between3And6,От 3 до 6 лет,"1.225 Продажи 1 Информационные технологии, инт...",NaN,standard,Стандарт,True,False,NaN,False,1391634.0,Брайт Солюшенз,https://api.hh.ru/vacancies?employer_id=1391634,True,https://hh.ru/employer/1391634,Разработка программного обеспечения\nСистемная...,False,open,...,NaN,NaN,True,Менеджер по работе с клиентами,NaN,1002,Минск,https://api.hh.ru/areas/1002,2016-03-10T20:57:08+0300,2016-03-10T20:57:08+0300,NaN,NaN,NaN,NaN,NaN,NaN,https://hh.ru/vacancy/16260126,https://hh.ru/applicant/vacancy_response?vacan...,NaN,NaN,NaN,full,Полная занятость,support,кофе team коммуникативные_навыки crm мониторин...
2,16260127,<p><strong>Требования:</strong></p> <p>1.Продв...,NaN,flexible,Гибкий график,True,False,noExperience,Нет опыта,"1.172 Начальный уровень, Мало опыта 1 Информац...",NaN,standard,Стандарт,True,False,NaN,False,2139835.0,Галкина А.А.,https://api.hh.ru/vacancies?employer_id=2139835,False,https://hh.ru/employer/2139835,NaN,False,open,...,NaN,RUR,True,Ведущий менеджер по персоналу,NaN,1,Москва,https://api.hh.ru/areas/1,2016-03-10T20:57:31+0300,2016-03-10T20:57:31+0300,NaN,NaN,NaN,NaN,NaN,NaN,https://hh.ru/vacancy/16260127,https://hh.ru/applicant/vacancy_response?vacan...,NaN,NaN,NaN,part,Частичная занятость,manager,гибкий_график желание_работать ms_excel официа...


(332460, 56)

In [273]:
df_raw = df_raw_raw.iloc[:]
len(df_raw)

332460

In [246]:
for col in ["id", "employer_id"]:
    print(col)
    df_raw[col] = df_raw[col].astype("Int64")

id
employer_id


C:\Users\d5767\AppData\Local\Temp\ipykernel_11892\39803199.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_raw[col] = df_raw[col].astype("Int64")


# Осмотр

In [110]:
df_raw[df_raw["prof_classes_found"].notna()]

,id,description,key_skills,schedule_id,schedule_name,accept_handicapped,accept_kids,experience_id,experience_name,specializations,contacts,billing_type_id,billing_type_name,allow_messages,premium,driver_license_types,accept_incomplete_resumes,employer_id,employer_name,employer_vacancies_url,employer_trusted,employer_alternate_url,employer_industries,response_letter_required,type_id,...,salary_gross,salary_currency,archived,name,insider_interview,area_id,area_name,area_url,created_at,published_at,address_city,address_street,address_building,address_description,address_lat,address_lng,alternate_url,apply_alternate_url,code,department_id,department_name,employment_id,employment_name,prof_classes_found,terms_found
0,16260123,<p>Современная и амбициозная российская компан...,UML\nUse Case Analysis\nнаписание технических ...,fullDay,Полный день,False,False,between1And3,От 1 года до 3 лет,"1.221 Программирование, Разработка 1 Информаци...",NaN,standard,Стандарт,True,False,NaN,False,707239,Embria,https://api.hh.ru/vacancies?employer_id=707239,True,https://hh.ru/employer/707239,"Интернет-компания (поисковики, платежные систе...",False,open,...,NaN,NaN,True,Системный аналитик,NaN,2,Санкт-Петербург,https://api.hh.ru/areas/2,2016-03-10T20:56:17+0300,2016-03-10T20:56:17+0300,NaN,NaN,NaN,NaN,NaN,NaN,https://hh.ru/vacancy/16260123,https://hh.ru/applicant/vacancy_response?vacan...,NaN,NaN,NaN,full,Полная занятость,analyst,техническое_образование конкурентная_зарплата ...
1,16260126,"<p><em><strong>Развивающаяся компания, успешно...",Активные продажи\nХолодные продажи\nВедение пе...,fullDay,Полный день,False,False,between3And6,От 3 до 6 лет,"1.225 Продажи 1 Информационные технологии, инт...",NaN,standard,Стандарт,True,False,NaN,False,1391634,Брайт Солюшенз,https://api.hh.ru/vacancies?employer_id=1391634,True,https://hh.ru/employer/1391634,Разработка программного обеспечения\nСистемная...,False,open,...,NaN,NaN,True,Менеджер по работе с клиентами,NaN,1002,Минск,https://api.hh.ru/areas/1002,2016-03-10T20:57:08+0300,2016-03-10T20:57:08+0300,NaN,NaN,NaN,NaN,NaN,NaN,https://hh.ru/vacancy/16260126,https://hh.ru/applicant/vacancy_response?vacan...,NaN,NaN,NaN,full,Полная занятость,support,кофе team коммуникативные_навыки crm мониторин...
2,16260127,<p><strong>Требования:</strong></p> <p>1.Продв...,NaN,flexible,Гибкий график,True,False,noExperience,Нет опыта,"1.172 Начальный уровень, Мало опыта 1 Информац...",NaN,standard,Стандарт,True,False,NaN,False,2139835,Галкина А.А.,https://api.hh.ru/vacancies?employer_id=2139835,False,https://hh.ru/employer/2139835,NaN,False,open,...,NaN,RUR,True,Ведущий менеджер по персоналу,NaN,1,Москва,https://api.hh.ru/areas/1,2016-03-10T20:57:31+0300,2016-03-10T20:57:31+0300,NaN,NaN,NaN,NaN,NaN,NaN,https://hh.ru/vacancy/16260127,https://hh.ru/applicant/vacancy_response?vacan...,NaN,NaN,NaN,part,Частичная занятость,manager,гибкий_график желание_работать ms_excel официа...
3,16260130,<p><strong>Обязанности:</strong></p> <p>1. Под...,NaN,flexible,Гибкий график,True,False,noExperience,Нет опыта,"1.172 Начальный уровень, Мало опыта 1 Информац...",NaN,standard,Стандарт,True,False,NaN,False,2139835,Галкина А.А.,https://api.hh.ru/vacancies?employer_id=2139835,False,https://hh.ru/employer/2139835,NaN,False,open,...,NaN,RUR,True,Специалист по подбору персонала,NaN,1,Москва,https://api.hh.ru/areas/1,2016-03-10T20:58:06+0300,2016-03-10T20:58:06+0300,NaN,NaN,NaN,NaN,NaN,NaN,https://hh.ru/vacancy/16260130,https://hh.ru/applicant/vacancy_response?vacan...,NaN,NaN,NaN,part,Частичная занятость,specialist,стабильная_заработная_плата внимательность sky...
4,16260135,<p><strong>Обязанности:</strong></p> <p>1. Под...,NaN,flexible,Гибкий график,True,False,noExperience,Нет опыта,"1.172 Начальный уровень, Мало опыта 1 Информац...",NaN,standard,Стандарт,True,False,NaN,False,2139835,Галкина А.А.,https://api.hh.ru/vacancies?employer_id=2139835,False,https://hh.ru/employer/2139835,NaN,False,open,...,NaN,RUR,True,Специалист по подбору персонала,NaN,2,

In [7]:
df_raw.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 56 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id                         100 non-null    Int64  
 1   description                100 non-null    object 
 2   key_skills                 36 non-null     object 
 3   schedule_id                100 non-null    object 
 4   schedule_name              100 non-null    object 
 5   accept_handicapped         100 non-null    bool   
 6   accept_kids                100 non-null    bool   
 7   experience_id              100 non-null    object 
 8   experience_name            100 non-null    object 
 9   specializations            100 non-null    object 
 10  contacts                   0 non-null      float64
 11  billing_type_id            100 non-null    object 
 12  billing_type_name          100 non-null    object 
 13  allow_messages             100 non-null    bool   


In [8]:
l_column_smoll_unique = []
for col in df_raw.columns:
    unique = df_raw[col].unique()
    if len(unique) < 20:
        print(col, "\t"*(4-((len(col)+1)//8)), len(unique), unique)
    else:
        print(col, "\t"*(4-((len(col)+1)//8)), len(unique))



id 				 100
description 			 83
key_skills 			 33
schedule_id 			 4 ['fullDay' 'flexible' 'remote' 'shift']
schedule_name 			 4 ['Полный день' 'Гибкий график' 'Удаленная работа' 'Сменный график']
accept_handicapped 		 2 [False  True]
accept_kids 			 1 [False]
experience_id 			 4 ['between1And3' 'between3And6' 'noExperience' 'moreThan6']
experience_name 		 4 ['От 1 года до 3 лет' 'От 3 до 6 лет' 'Нет опыта' 'Более 6 лет']
specializations 		 72
contacts 			 1 [nan]
billing_type_id 		 3 ['standard' 'free' 'standard_plus']
billing_type_name 		 3 ['Стандарт' 'Бесплатная' 'Стандарт плюс']
allow_messages 			 2 [ True False]
premium 			 1 [False]
driver_license_types 		 1 [nan]
accept_incomplete_resumes 	 2 [False  True]
employer_id 			 71
employer_name 			 71
employer_vacancies_url 		 71
employer_trusted 		 2 [True False]
employer_alternate_url 		 71
employer_industries 		 36
response_letter_required 	 2 [False  True]
type_id 			 1 ['open']
type_name 			 1 ['Открытая']
has_test 			 1 [False]
r

In [9]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 56 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id                         100 non-null    Int64  
 1   description                100 non-null    object 
 2   key_skills                 36 non-null     object 
 3   schedule_id                100 non-null    object 
 4   schedule_name              100 non-null    object 
 5   accept_handicapped         100 non-null    bool   
 6   accept_kids                100 non-null    bool   
 7   experience_id              100 non-null    object 
 8   experience_name            100 non-null    object 
 9   specializations            100 non-null    object 
 10  contacts                   0 non-null      float64
 11  billing_type_id            100 non-null    object 
 12  billing_type_name          100 non-null    object 
 13  allow_messages             100 non-null    bool   


# Постройка графа

Нода вакансия (V):
id 				 332460
name 				 92470
description 			 230806
key_skills 			 42173 #TODO: возможно в отдельные ноды
schedule_id 			 5 ['fullDay' 'flexible' 'remote' 'shift' 'flyInFlyOut']
schedule_name 			 5 ['Полный день' 'Гибкий график' 'Удаленная работа' 'Сменный график' 'Вахтовый метод']
accept_handicapped 		 2 [False  True]
accept_kids 			 1 [False]
experience_id 			 4 ['between1And3' 'between3And6' 'noExperience' 'moreThan6']
contacts 			 1 [nan]
billing_type_id 		 3 ['standard' 'free' 'standard_plus']
allow_messages 			 2 [ True False]
premium 			 1 [False]
driver_license_types 		 1 [nan]
accept_incomplete_resumes 	 2 [False  True]
response_letter_required 	 2 [False  True]
type_id 			 3 ['open' 'direct' 'anonymous']
salary_from 			 1130
salary_to 			 971
salary_gross 			 1 [nan]
salary_currency 		 11 [nan 'RUR' 'BYR' 'UAH' 'USD' 'KZT' 'EUR' 'AZN' 'UZS' 'KGS' 'GEL']
created_at 			 285317
published_at 			 285317
employment_id 			 5 ['full' 'part' 'project' 'probation' 'volunteer']


Нода Работодателя (employer):
employer_id 			 41521
employer_name 			 40912
employer_trusted 		 3 [True False nan]


Нода специализации (special):
specializations(list) 		 60875

Нода Индустрия (ind)
employer_industries(list) 		 6963

Нода область (area)
area_id 			 2144
area_name 			 2145

Нода адреса (address)
address_city 			 1147
address_street 			 7390
address_building 		 4333
address_description 		 1 [nan]
address_lat 			 20810
address_lng 			 20816

Нода департамента (depar):
department_id 			 144
department_name 		 148

Нода проф классов (prof) 
prof_classes_found 		 706

Нода найденных определений (terms)
terms_found 			 171934


## Подготовка списков

In [247]:
__COLORS_NODES__ = {
    "vacancies": "#62e1ff",
    "employer": "#d77833",
    "specializations": "#d83377",
    "employer_industries": "#d833d8",
    "areas": "#f3e00d", 
    "city": "#8d33d8",
    "address": "#d85fd8",
    "classes": "#a3bfdf",
    "terms": "#d8f3d8"
}

### Список вакансий

In [248]:
df_nodes_vacancies = df_raw.loc[:,["id", "name", "description", "key_skills", 
                                   "schedule_id", "schedule_name", "accept_handicapped", 
                                   "accept_kids", "experience_id", "contacts", "billing_type_id", 
                                   "allow_messages", "premium", "driver_license_types", 
                                   "accept_incomplete_resumes", "response_letter_required", 
                                   "type_id", "salary_from", "salary_to", "salary_gross", 
                                   "salary_currency", "created_at", "published_at", "employment_id"]]
df_nodes_vacancies = df_nodes_vacancies.rename({"name":"label"}, axis="columns")
df_nodes_vacancies["id"] = df_nodes_vacancies["id"].astype(int)
df_nodes_vacancies = df_nodes_vacancies.set_index("id")
df_nodes_vacancies.index = "vacan_" + df_nodes_vacancies.index.astype(str)
df_nodes_vacancies["color"] = __COLORS_NODES__["vacancies"]
df_nodes_vacancies["type"] = "vacancies"

### Список работодателей

In [249]:
df_nodes_employer = df_raw.loc[:,["employer_id", "employer_name", "employer_trusted", "employer_industries"]]\
    .drop_duplicates(["employer_id"]) # TODO: Короче, ничто не уникально! 
df_nodes_employer = df_nodes_employer.set_index("employer_id")
df_nodes_employer.index = "employer_" + df_nodes_employer.index.astype(str)
df_nodes_employer["color"] = __COLORS_NODES__["employer"]
df_nodes_employer["type"] = "employer"


In [250]:
df_edges_employer = df_raw.loc[:,["id", "employer_id"]].rename({"id":"source", "employer_id":"target"},axis=1)
df_edges_employer["source"] = "vacan_" + df_edges_employer["source"].astype(str)
df_edges_employer["target"] = "employer_" + df_edges_employer["target"].astype(str)

### Список специализаций

In [251]:
df_nodes_specializations = df_raw.loc[:,["specializations"]]
df_nodes_specializations["specializations"] = df_nodes_specializations["specializations"].str.split("\n")
df_nodes_specializations = df_nodes_specializations.explode("specializations")
df_nodes_specializations = df_nodes_specializations.drop_duplicates().set_index("specializations")
df_nodes_specializations.index = "special_" + df_nodes_specializations.index.astype(str)
df_nodes_specializations["color"] = __COLORS_NODES__["specializations"]
df_nodes_specializations["type"] = "specializations"

In [252]:
df_edges_specializations = df_raw.loc[:,["id", "specializations"]].rename({"id":"source", "specializations":"target"},axis=1)
df_edges_specializations["target"] = df_edges_specializations["target"].str.split("\n")
df_edges_specializations = df_edges_specializations.explode("target")
df_edges_specializations["target"] = "special_" + df_edges_specializations["target"].astype(str)
df_edges_specializations["source"] = "vacan_" + df_edges_specializations["source"].astype(str)


### Список индустрий

In [253]:
df_nodes_employer_industries = df_raw.loc[:,["employer_industries"]].drop_duplicates()
df_nodes_employer_industries["employer_industries"] = df_nodes_employer_industries["employer_industries"].str.split("\n")
df_nodes_employer_industries = df_nodes_employer_industries.explode("employer_industries")
df_nodes_employer_industries = df_nodes_employer_industries.drop_duplicates().set_index("employer_industries")
df_nodes_employer_industries["label"] = df_nodes_employer_industries.index.astype(str)
df_nodes_employer_industries.index = "ind_" + df_nodes_employer_industries.index.astype(str)
df_nodes_employer_industries["color"] = __COLORS_NODES__["employer_industries"]
df_nodes_employer_industries["type"] = "employer_industries"

In [254]:
df_edges_employer_industries = df_raw.loc[:,["employer_id", "employer_industries"]].rename({"employer_id":"source", "employer_industries":"target"},axis=1)
df_edges_employer_industries["target"] = df_edges_employer_industries["target"].str.split('\n')
df_edges_employer_industries = df_edges_employer_industries.explode("target")
df_edges_employer_industries["target"] = "ind_" + df_edges_employer_industries["target"].astype(str)
df_edges_employer_industries["source"] = "employer_" + df_edges_employer_industries["source"].astype(str)

### Список областей

In [255]:
df_nodes_areas= df_raw.loc[:,["area_id", "area_name"]].drop_duplicates().set_index("area_id")
df_nodes_areas.index = "area_" + df_nodes_areas.index.astype(str)
df_nodes_areas["color"] = __COLORS_NODES__["areas"]
df_nodes_areas["label"] = df_nodes_areas["area_name"]
df_nodes_areas["type"] = "areas"

In [256]:
df_edges_employer_areas = df_raw.loc[:,["employer_id", "area_id"]].rename({"employer_id":"source", "area_id":"target"},axis=1).drop_duplicates()
df_edges_employer_areas["target"] = "area_" + df_edges_employer_areas["target"].astype(str)
df_edges_employer_areas["source"] = "employer_" + df_edges_employer_areas["source"].astype(str)


### Список адресов

In [257]:
df_nodes_address = df_raw.loc[:,["id", "address_city", "address_street", "address_building", "address_lat", "address_lng"]]\
    .groupby(["address_city", "address_street", "address_building", "address_lat", "address_lng"]).agg({
        "id":list
    }).reset_index().reset_index(names="id_address_node")
df_nodes_address["id_address_node"] = "address_" + df_nodes_address["id_address_node"].astype(str)
df_nodes_address["label"] = df_nodes_address["address_city"] + ", " +\
    df_nodes_address["address_street"] + " " +\
    df_nodes_address["address_building"]

df_nodes_city = df_nodes_address.groupby(["address_city"]).agg({
    "id_address_node":list
}).reset_index(names="city_name").reset_index(names="id_city_node")
df_nodes_city["id_city_node"] = "city_" + df_nodes_city["id_city_node"].astype(str)
df_nodes_city["label"] = df_nodes_city["city_name"].astype(str)


df_edges_address = df_nodes_address.loc[:, ["id_address_node", "id"]]\
    .explode("id")\
    .rename({"id":"source", "id_address_node":"target"},axis=1)
df_edges_address["source"] = "vacan_" + df_edges_address["source"].astype(str)


df_edges_city = df_nodes_city.loc[:, ["id_address_node", "id_city_node"]]\
    .explode("id_address_node")\
    .rename({"id_city_node":"source", "id_address_node":"target"},axis=1)


df_nodes_city = df_nodes_city.drop(columns=["id_address_node"]).set_index("id_city_node")
df_nodes_address = df_nodes_address.drop(columns=["address_city", "id"]).set_index("id_address_node")

df_nodes_city["type"] = "city"
df_nodes_address["type"] = "address"
df_nodes_city["color"] = __COLORS_NODES__["city"]
df_nodes_address["color"] = __COLORS_NODES__["address"]


### Список профессиональных областей

In [258]:
df_nodes_classes= df_raw.loc[:,["prof_classes_found"]].drop_duplicates().set_index("prof_classes_found")
df_nodes_classes["label"] = df_nodes_classes.index
df_nodes_classes.index = "classes_" + df_nodes_classes.index.astype(str)
df_nodes_classes["type"] = "classes"
df_nodes_classes["color"] = __COLORS_NODES__["classes"]


In [259]:
df_edges_classes = df_raw.loc[:,["id", "prof_classes_found"]].rename({"id":"source", "prof_classes_found":"target"},axis=1).drop_duplicates()
df_edges_classes["target"] = "classes_" + df_edges_classes["target"].astype(str)
df_edges_classes["source"] = "vacan_" + df_edges_classes["source"].astype(str)

### Список найденных определений

In [260]:
df_nodes_terms= df_raw.loc[:,["terms_found"]].drop_duplicates().set_index("terms_found")
df_nodes_terms["label"] = df_nodes_terms.index
df_nodes_terms.index = "terms_" + df_nodes_terms.index.astype(str)
df_nodes_terms["type"] = "terms"
df_nodes_terms["color"] = __COLORS_NODES__["terms"]


In [261]:
df_edges_terms = df_raw.loc[:,["id", "terms_found"]].rename({"id":"source", "terms_found":"target"},axis=1).drop_duplicates()
df_edges_terms["target"] = "terms_" + df_edges_terms["target"].astype(str)
df_edges_terms["source"] = "vacan_" + df_edges_terms["source"].astype(str)

# Загрузка данных


In [267]:
list_df_nodes = {
    "vacancies": df_nodes_vacancies,
    "employer": df_nodes_employer,
    "specializations": df_nodes_specializations,
    "employer_industries": df_nodes_employer_industries,
    "employer_areas": df_nodes_areas,
    "city": df_nodes_city,
    "address": df_nodes_address,
    "classes": df_nodes_classes,
    "terms": df_nodes_terms
}

list_df_edges = {
    "employer": df_edges_employer,
    "specializations": df_edges_specializations,
    "employer_industries": df_edges_employer_industries,
    "employer_areas": df_edges_employer_areas,
    "city": df_edges_city,
    "address": df_edges_address,
    "classes": df_edges_classes,
    "terms": df_edges_terms
}

__RADIUS__ = 4000

group_positions = {
    "vacancies": (0, 0),
    "employer": (0, __RADIUS__),
    "specializations": (__RADIUS__, 0),
    "employer_industries": (-__RADIUS__, __RADIUS__),
    "employer_areas": (__RADIUS__, __RADIUS__),
    "city": (-__RADIUS__, -__RADIUS__),
    "address": (-__RADIUS__, 0),
    "classes":(0, -__RADIUS__),
    "terms": (__RADIUS__, -__RADIUS__)
}


In [268]:
def add_edges_from_dataframe(G, df, source_col='source', target_col='target'):
    edges = []
    for index, row in df.iterrows():
        # Извлекаем начальный и конечный узлы
        source = row[source_col]
        target = row[target_col]
        
        # Извлекаем остальные атрибуты
        attributes = row.drop([source_col, target_col]).to_dict()
        
        # Создаем кортеж с ребром и атрибутами
        edges.append((source, target, attributes))
    
    # Добавляем рёбра в граф
    G.add_edges_from(edges)

In [269]:
G = nx.DiGraph()

for df_name, df in list_df_nodes.items():
    time_start = time.time()
    print(f"add nodes {df_name}...", end="\r")
    # ser_title = df.drop(columns=["color", "label"], errors="ignore")\
    ser_title = df\
        .astype(str)\
        .apply(lambda x: "\n".join([f"{name}: {valve}" for name, valve in x.to_dict().items()]), 
               axis=1)
    ser_title.name = "title"
    G.add_nodes_from((n, dict(d)) for n, d in df.assign(title=ser_title).iterrows())
    print(f"add nodes {df_name}: done load {len(df)} nodes in {time.time()-time_start:.2f} sec")
print("total nodes:", len(G), "\n")

for df_name, df in list_df_edges.items():
    time_start = time.time()
    print(f"add edges {df_name}...", end="\r")
    add_edges_from_dataframe(G, df)
    print(f"add edges {df_name}: done load {len(df)} edges in {time.time()-time_start:.2f} sec")
print("total edges:", len(G.edges()), "\n")


for df_name, df in list_df_nodes.items():
    time_start = time.time()
    print(f"add center {df_name}...", end="\r")
    G.add_node(df_name, 
               shape="star", 
               physics=False, 
               x=group_positions[df_name][0],
               y=group_positions[df_name][1],
               color="red",
               hidden=True
               )
    df_new_adges = pd.DataFrame({
        "target": df_name,
        "source": df.index
        })
    
    add_edges_from_dataframe(G, df_new_adges)
    print(f"add center {df_name}: done in {time.time()-time_start:.2f} sec")


add nodes vacancies: done load 500 nodes in 0.17 sec
add nodes employer: done load 374 nodes in 0.06 sec
add nodes specializations: done load 165 nodes in 0.04 sec
add nodes employer_industries: done load 163 nodes in 0.02 sec
add nodes employer_areas: done load 128 nodes in 0.02 sec
add nodes city: done load 45 nodes in 0.01 sec
add nodes address: done load 148 nodes in 0.03 sec
add nodes classes: done load 64 nodes in 0.01 sec
add nodes terms: done load 431 nodes in 0.05 sec
total nodes: 2018 

add edges employer: done load 500 edges in 0.31 sec
add edges specializations: done load 1801 edges in 0.89 sec
add edges employer_industries: done load 925 edges in 1.38 sec
add edges employer_areas: done load 449 edges in 0.19 sec
add edges city: done load 148 edges in 0.08 sec
add edges address: done load 166 edges in 0.10 sec
add edges classes: done load 500 edges in 0.20 sec
add edges terms: done load 500 edges in 0.20 sec
total edges: 4794 

add center vacancies: done in 0.21 sec
add cen

In [270]:
graf_to_show = G


net = Network(filter_menu=True) # создаём объект графа
net.show_buttons(filter_=["physics"])
net.from_nx(graf_to_show)
net.toggle_physics(False)

net.show('graph.html', local=False)

In [266]:
from plyer import notification
# Send notification
notification.notify(
    title="Код выполнен",
    message="Процесс выполнения кода завершён!",
    app_name="Твой код",
    timeout=20
    )